<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Membrane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##MemProtMD Membrane Builder

[MemProtMD](https://doi.org/10.1016/j.str.2015.05.006) membrane protein association protocol for allowing a protein to self-associate with a preformed lipid membrane.

Set-up for a [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) lipid membrane simulation,constructed using [Insane](https://doi.org/10.1021/acs.jctc.5b00209). The system is briefly equilibrated using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

Inspiration for this Google Colab was taken from the amazing work of the [ColabFold](https://github.com/sokrypton/ColabFold) team.

Change the settings below and click **Runtime → Run all**.  You will be prompted to upload a Soluble Protein PDB file to simulate.

In [ ]:
#@title Define Settings
#@markdown ---
#@markdown #### Set Membrane Type:
#@markdown #### Enter lipid types and their quantities. For example:
#@markdown #### POPE 4 POPG 1
MembraneType =  "POPE 4 POPG 1" #@param {type:"string"}

Repeats = 1
#@markdown ---
#@markdown #### Set Box Dimensions:
Box_Width = 12 #@param {type:"slider", min:8, max:30, step:1}
Box_Length = 12 #@param {type:"slider", min:12, max:50, step:1}
#@markdown ---

working_dir = '/content/'

In [ ]:
#@title Upload PDB coordinate files
import os
import sys

cube = [Box_Width,Box_Width,Box_Length]

x = Box_Width
y = Box_Width
z = Box_Length

# Split the input string into a list
input_list = MembraneType.split()

# Initialize the lipid variable
lipid = ""

# Iterate over the list in pairs
for i in range(0, len(input_list), 2):
    lipid_type = input_list[i]
    quantity = input_list[i+1]
    lipid += f"-l {lipid_type}:{quantity} "

# Strip the trailing space
lipid = lipid.strip()

#if MembraneType == "POPC":
#  lipid = '-l POPC:1'
#elif MembraneType == "POPE:POPG":
#  lipid = '-l POPE:7 -l POPG:3'
#elif MembraneType == "POPE:POPG:CARDIOLIPIN":
#  lipid = '-l POPE:7 -l POPG:2 -l CARD:1'

!python3 -m pip install py3dmol
!python3 -m pip install colorama

import py3Dmol
from colorama import Fore
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')

In [ ]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/cg2at/"):
  !apt-get update -y
  !apt-get install dssp
  !python3 -m pip  install wget
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install gromacswrapper==0.8.3
  !python3 -m pip install MDAnalysis
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  %mv /content/usr/local/gromacs/ /usr/local/
  !ln -s /usr/local/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil
import wget
from random import randrange

In [ ]:
#@title Set-up Coarse-Grained Membrane Protein Systems
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini_v300/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')
with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n\n[ system ]\n[ molecules ]\n')

for rep in range(1,Repeats+1):
  rep=str(rep)
  os.chdir(working_dir)
  os.system('python /content/insane3.py '+ lipid +' -salt 0.15 -sol W -o CG-system.gro -p topol.top -center -rotate random -ring -x %s -y %s -z %s' % (x, y, z))

  replacements = {'NA+':'NA', 'CL-':'CL', '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
  lines = []
  with open('topol.top') as infile:
      for line in infile:
          for src, target in replacements.items():
              line = line.replace(src, target)
          lines.append(line)
  with open('topol.top', 'w') as outfile:
      for line in lines:
          outfile.write(line)

  os.system('cp topol.top topol'+rep+'.top')

  gromacs.grompp(f='em.mdp',o='em'+ rep +'.tpr',c='CG-system.gro',maxwarn=10,backup=False,v=True)
  gromacs.mdrun(deffnm='em'+ rep, c='CG-system'+ rep +'.pdb',backup=False)
  gromacs.trjconv(f='CG-system'+ rep +'.pdb', o='CG-system'+ rep +'.pdb', pbc='mol', s='em'+ rep +'.tpr', conect=True, input='0',backup=False)

  os.makedirs('EQ'+rep, exist_ok=True)
  gromacs.make_ndx(f='CG-system'+ rep +'.pdb', o='index'+ rep +'.ndx', input=('del 0-100','rPOP*|rCARD','!0','name 0 Lipid','name 1 SOL_ION','q'),backup=False)

  with open('cgequil1.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = LIPID SOL_ION\ntau_t  = 1.0 1.0\nref_t  = 310 310\nPcoupl  = no\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')
  with open('cgequil2.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = LIPID SOL_ION\ntau_t  = 1.0 1.0\nref_t  = 310 310\nPcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 4.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = no\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = yes\nlincs_order  = 4\nlincs_warnangle = 30\n')
  with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 50000000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = LIPID SOL_ION\ntau_t  = 1.0 1.0\nref_t  = 310 310\nPcoupl  = c-rescale\nPcoupltype  = semiisotropic\ntau_p  = 4.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = no\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')


  gromacs.grompp(f='cgequil1.mdp',o='EQ'+ rep +'/eq1',c='CG-system'+ rep +'.pdb',maxwarn=10, n='index'+ rep +'.ndx',backup=False)
  os.chdir(working_dir + '/EQ' + rep)
  gromacs.mdrun(deffnm='eq1',backup=False, v=True)
  os.chdir(working_dir)

  gromacs.grompp(f='cgequil2.mdp',o='EQ'+ rep +'/eq2',c='EQ'+ rep +'/eq1.gro',maxwarn=10, n='index'+ rep +'.ndx',backup=False)
  os.chdir(working_dir + '/EQ' + rep)
  gromacs.mdrun(deffnm='eq2',backup=False, v=True)
  os.chdir(working_dir)

  os.makedirs('MD'+rep, exist_ok=True)
  gromacs.grompp(f='cgmd.mdp',o='MD'+ rep +'/md',c='EQ'+ rep +'/eq2.gro',maxwarn=10, n='index'+ rep +'.ndx',backup=False)


In [ ]:
#@title Convert back to Atomistic
os.chdir(working_dir)
shutil.rmtree('Atomistic', ignore_errors=True)
os.chdir(working_dir + 'EQ1/')
shutil.rmtree('CG2AT', ignore_errors=True)

# Convert CG to Atomistic
os.system('/content/cg2at/cg2at -group all -w tip3p -c eq2.gro -loc CG2AT -ff charmm36-jul2020-updated -fg martini_3-0_charmm36')

os.chdir(os.path.join(working_dir, 'EQ1/CG2AT/FINAL/'))

# Run GROMACS commands
gromacs.grompp(f='../../../em.mdp', o='final.tpr', c='final_cg2at_de_novo.pdb', p='topol_final.top',maxwarn=10)

# Copy files
shutil.copytree(working_dir + 'EQ1/CG2AT/FINAL/', working_dir + 'Atomistic/')

os.chdir(os.path.join(working_dir, 'Atomistic'))
gromacs.editconf(f='final.tpr',o='System.pdb')


# Rename files
#os.rename(os.path.join(working_dir, 'MD', 'md.pdb'), os.path.join(working_dir, f"{name}-cgmd.pdb"))
#os.rename(os.path.join(working_dir, 'MD', 'md.tpr'), os.path.join(working_dir, 'cgmd.tpr'))
#os.rename(os.path.join(working_dir, 'memembed.pdb'), os.path.join(working_dir, f"{name}-memembed.pdb"))

# Create index file
os.replace(os.path.join(working_dir, 'Atomistic', 'topol_final.top'),
           os.path.join(working_dir, 'Atomistic', 'topol.top'))
gromacs.make_ndx(f='System.pdb', o='index.ndx', input=('del 2-40', 'rSOL|rNA*|rCL*','1|2','0&!3','del 3','name 2 water_and_ions','name 3 Lipid','q'),backup=False)

# Edit topology file
input_file = working_dir + 'Atomistic/topol.top'
output_file = input_file

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    filedata = infile.read()
    filedata = filedata.replace('../FINAL/', '')
    outfile.write(filedata)

prod = 'https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/500ns.mdp'
equil = 'https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/10ns-pr.mdp'
filename = wget.download(prod)
filename = wget.download(equil)

In [ ]:
#@title Visualise Final systems
for rep in range(1,Repeats+1):
  print(Fore.BLUE + '\nRepeat ' + str(rep) + '\n')
  rep=str(rep)
  mview = py3Dmol.view(width=800,height=400)
  mol1 = open(working_dir + 'EQ'+rep +'/eq2.gro', 'r').read()
  mview.addModel(mol1,'gro')
  mview.setStyle({'cartoon':{'color':'spectrum'}})
  mview.setStyle({'atom':'PO1','atom':'PO2','atom':'PO3','atom':'PO4'},{'sphere':{}})
  mview.setStyle({'atom':'BB'},{'sphere':{}})
  mview.setBackgroundColor('0xffffff')
  mview.zoomTo()
  mview.show()
  for file in glob.glob(r'#*'):
      os.remove(file)

os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [ ]:
#@title Download Zip
os.chdir(working_dir)
shutil.rmtree('MD', ignore_errors=True)
try:
    os.remove('em*.trr')
except OSError:
    pass
try:
    os.remove('em*.tpr')
except OSError:
    pass
try:
    os.remove('em*.log')
except OSError:
    pass
try:
    os.remove('protein.pdb')
except OSError:
    pass
try:
    os.remove('molecule_0.itp')
except OSError:
    pass
try:
    os.remove('mdout.mdp')
except OSError:
    pass
try:
    os.remove('aligned.gro')
except OSError:
    pass
try:
    os.remove('protein-cg.top')
except OSError:
    pass
os.chdir('/content/')
os.system('zip -r Atomistic.zip Atomistic')
files.download('Atomistic.zip')